# Convert checkpoint GPKG to geoparquet 

In [ ]:
import geopandas as gpd

In [ ]:
%%bash

wget -nc https://www.sciencebase.gov/catalog/file/get/67075e6bd34e969edc59c3e7\?f\=__disk__80%2F12%2F9e%2F80129e86d18461ed921b288f13e08c62e8590ffb -O checkpoints.zip
unzip checkpoints.zip

In [ ]:
ls -lth Checkpoints_3DEP_2004_2025.gpkg

In [ ]:
!gdal vector info Checkpoints_3DEP_2004_2025.gpkg

In [ ]:
# Convert with geopandas
gf = gpd.read_file('Checkpoints_3DEP_2004_2025.gpkg')
gf.head()

In [ ]:
# STAC-map requires parquet with 'id' and 'datetime' I think
gf = gf.rename(columns=dict(collection_date='datetime', unique_identifier='id'))

# Also maybe requires 2D geometries?
gf['geometry'] = gf.geometry.force_2d()

gf = gf.drop(columns='OBJECTID')

In [ ]:
gf.head()

In [ ]:
gf.to_parquet('checkpoints_3dep_2004_2025.parquet',
              schema_version='1.1.0',
              geometry_encoding='WKB', # or 'geoarrow'
              index=False,
              write_covering_bbox=True)

In [ ]:
# TODO: compare w/ GDAL...
#!gdal vector convert --overwrite --lco WRITE_COVERING_BBOX=YES -i Checkpoints_3DEP_2004_2025.gpkg -o checkpoints_3dep_2004_2025.parquet